# Import Package

In [ ]:
import csv
from socket import MSG_MCAST
from urllib.parse import quote, quote_plus
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# 날짜 지정

In [ ]:
strDate = '20200901' # 시작 날짜
endDate = '20200930' # 종료 날짜

# CSV 저장

In [ ]:
filename = f"daum_news_{strDate}-{endDate}.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

# 유저 에이전트 지정
> [WhatismyUserAgent(ctrl+🖱 후 확인)](https://www.whatismybrowser.com/detect/what-is-my-user-agent)

In [ ]:
headers = {"User-Agent" : "개인 유저 에이전트"}

# 반복문에 들어갈 날짜 지정

In [ ]:
dt_index = pd.date_range(start=strDate, end=endDate)
dt_list = dt_index.strftime("%Y%m%d").tolist()

# 스크래핑 시작

In [ ]:
for i in dt_list:
    print('날짜',i)
    page_url = f"https://news.daum.net/breakingnews/?page=10000&regDate={i}"
    page =requests.get(page_url, headers=headers)
    page_soup = BeautifulSoup(page.text, "lxml")
    last_page = page_soup.find("em", attrs="num_page").get_text()
    lastPage_num = re.sub(r'[^0-9]','',last_page)
    # print(lastPage_num)

    for j in range(1, int(lastPage_num)+1):
        main_url = f"https://news.daum.net/breakingnews/?page={j}&regDate={i}" # url 입력
        res = requests.get(main_url, headers=headers)
 
        if res.status_code == 200:
            print(i, int(lastPage_num), '중' ,j,'page',round(j/int(lastPage_num)*100, 2),'%', main_url, 'status:',res.status_code)
            soup = BeautifulSoup(res.text, "lxml") # soup으로 저장
            main = soup.find("ul", attrs={"class":"list_news2 list_allnews"})
            news = main.find_all("strong", attrs={"class":"tit_thumb"})
            cnt = 0
            for new in news:
                urls = new.select_one("a")["href"]# 페이지에 나와있는 뉴스 URL 변수 입력
                # print(urls)
                result = requests.get(urls, headers=headers)         # request 로 다시 개별 뉴스 접속
                if result.status_code == 200:
                    news_soup = BeautifulSoup(result.text, "lxml")
                    # 뉴스 제목, 발행시간, 기사본문 저장
                    title = news_soup.find("h3", attrs={"tit_view"}).get_text().strip()
                    pubdate = news_soup.find("span", attrs={"num_date"}).get_text().strip()
                    text = news_soup.find("div", attrs={"news_view"}).get_text().strip()
                    cnt += 1
                    # print(j,'of',cnt,'번째 기사')
                    # print(i,j,'of',cnt,'번째 기사', urls,'status:', result.status_code)
                    writer.writerow([cnt, title, pubdate, urls, text])
                else:
                    print(i,j,'of',cnt,'번째 기사','error_code :',result.status_code, urls)
                    pass
                   
        else:
            print(i,'page : ',j,'error_code :',res.status_code, main_url)
            pass